In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [24]:
## hyperparameter

device = 'cuda' if torch.cuda.is_available() else 'cpu'


batch_size =10
learning_rate = 1e-3
num_epochs =25


In [25]:
## Transform

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((132,132)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_data = datasets.ImageFolder('.\\archive\\Training_Crop_Aug\\',  transform = transform)
train_dataloader = torch.utils.data.DataLoader(train_data,batch_size = batch_size,shuffle = True)

test_data = datasets.ImageFolder('.\\archive\\Testing_Crop\\',  transform = transform)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size = batch_size,shuffle = True)

for num, value in enumerate(test_data):
    data, label =value
    #print(num,label)
    #print(data.shape)


In [26]:
## CNN

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,8,5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32*30*30,4, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        '''
        self.dropout = nn.Dropout(0.3)
        self.conv1 = nn.Conv2d(3,8,5,stride=1)
        self.conv2 = nn.Conv2d(8,32,5,stride = 1)
        self.conv3 = nn.Conv2d(32,128,3,stride = 1,padding = 1)
        self.pool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(128*15*15,1024)
        self.fc2 = nn.Linear(1024,128)
        self.fc3 = nn.Linear(128,4)
        '''
    def forward(self,x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0),-1)
        #print(out.shape)
        out = self.fc(out)
        return out

In [27]:
model = CNN().to(device)
A = (torch.Tensor(10,3,132,132)).to(device)
test_out = model(A)
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=28800, out_features=4, bias=True)
)

In [28]:
criterion = nn.CrossEntropyLoss().to(device)

#Use Adam optimizer(We tried a lot of optimizers but results are similar)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [29]:


def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [30]:

total_batch = len(train_dataloader)

for epoch in range(num_epochs):
    avg_cost = 0
    for images, labels in train_dataloader:
        #Push tensors to GPU
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(images)

        cost = criterion(hypothesis, labels)
        cost.backward()      
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    #loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

[Epoch:1] cost = 0.9293234348297119
[Epoch:2] cost = 0.5603933930397034
[Epoch:3] cost = 0.36964476108551025
[Epoch:4] cost = 0.23348824679851532
[Epoch:5] cost = 0.15293967723846436
[Epoch:6] cost = 0.09515132009983063
[Epoch:7] cost = 0.07054746896028519
[Epoch:8] cost = 0.03597705066204071
[Epoch:9] cost = 0.04078853130340576
[Epoch:10] cost = 0.07069267332553864
[Epoch:11] cost = 0.046667102724313736
[Epoch:12] cost = 0.045311667025089264
[Epoch:13] cost = 0.024574359878897667
[Epoch:14] cost = 0.034348439425230026
[Epoch:15] cost = 0.025174809619784355
[Epoch:16] cost = 0.04557359218597412
[Epoch:17] cost = 0.0355619341135025
[Epoch:18] cost = 0.05225713551044464
[Epoch:19] cost = 0.01356074120849371
[Epoch:20] cost = 0.0014079648535698652
[Epoch:21] cost = 0.00024340780510101467
[Epoch:22] cost = 0.0001209055189974606
[Epoch:23] cost = 8.080714178504422e-05
[Epoch:24] cost = 6.168738036649302e-05
[Epoch:25] cost = 4.7072855522856116e-05


In [32]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 72.84263959390863 %


In [209]:
###### need to check single img ######
"""
import cv2
import matplotlib.image as mpimg
im = cv2.imread('C:\\Users\\jk121\\Desktop\\temp\\archive\\Training\\p(1).jpg')



def test_one_image(I, model):
    '''
    I - 28x28 uint8 numpy array
    '''

    # test phase
    model.eval()

    # convert image to torch tensor and add batch dim
    batch = torch.tensor(I / 255).unsqueeze(0)

    # We don't need gradients for test, so wrap in 
    # no_grad to save memory
    with torch.no_grad():
        batch = batch.to(device)

        # forward propagation
        output = model( batch )

        # get prediction
        output = torch.argmax(output, 1)

    return output
"""